In [8]:
import time

import joblib
import pandas as pd
import requests
from kafka import KafkaProducer
from sklearn.linear_model import LinearRegression

In [9]:
def get_bitcoin_data():
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    params = {"vs_currency": "usd", "days": "30"}
    response = requests.get(url, params)
    data = response.json()
    prices = data['prices']
    df = pd.DataFrame(prices, columns=["timestamp", "price"])
    df["timestamp"] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

In [11]:
def train_model():
    df = get_bitcoin_data()
    if df is None or df.empty:
        print("Erro: Dados do Bitcoin não disponíveis.")
        return None

    df['price_previous'] = df['price'].shift(1)
    df = df.dropna()
    X = df[['price_previous']]
    y = df['price']
    
    model = LinearRegression()
    model.fit(X, y)
    
    joblib.dump(model, 'bitcoin_model.pkl')
    
    print("✅ Modelo treinado com sucesso")
    return "Modelo treinado com sucesso"

In [4]:
def predict_price():
    df = get_bitcoin_data()
    
    if df.empty:
        raise ValueError("Erro: Dados do Bitcoin não foram coletados corretamente.")
    
    model = joblib.load('bitcoin_model.pkl')
    
    last_price = df['price'].iloc[-1]
    predicted_price = model.predict([[last_price]])[0]  # Pegamos apenas o primeiro valor da previsão
    
    # Formatando para duas casas decimais
    predicted_price_formatted = round(float(predicted_price), 2)
    
    print(f"💲Preço previsto para o próximo dia: {predicted_price_formatted}")
    
    return predicted_price_formatted

In [12]:
get_bitcoin_data()

,timestamp,price
0,2025-01-07 23:01:54.139,96966.665012
1,2025-01-08 00:03:56.480,96952.098868
2,2025-01-08 01:00:01.762,97121.198531
3,2025-01-08 02:00:06.054,96652.024304
4,2025-01-08 03:03:41.727,96866.906198
...,...,...
714,2025-02-06 19:02:56.212,96340.129989
715,2025-02-06 20:04:01.843,95967.150040
716,2025-02-06 21:03:56.060,96900.668886
717,2025-02-06 22:01:12.092,96784.585045


In [13]:
train_model()

✅ Modelo treinado com sucesso


'Modelo treinado com sucesso'

In [14]:
predict_price()

💲Preço previsto para o próximo dia: 96813.34


/home/policarpo/Projetos/Pessoal/pipeline-dados-btc/app/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


96813.34